In [1]:
import nibabel as nib
from glob import glob
import nrrd
import numpy as np
from tqdm import tqdm
from scipy import ndimage
import torchio as tio
import os

In [2]:
city = 'Beijing_Zang'
vent = '1'
size = 64
MASK_DIR = f'../dataset/{city}/6classes/Ventricles{vent}'
mask_paths = sorted(glob(f'{MASK_DIR}/*.nii.gz'))
subjects = [tio.Subject(label=tio.LabelMap(mask_path)) for mask_path in mask_paths]

In [3]:
resample = tio.Compose([
    tio.Resample(1),
    tio.CropOrPad(size, mask_name='label'),
])

dataset = tio.SubjectsDataset(
    subjects, 
    transform=resample
)

In [4]:
out_dir = f'../dataset/{city}/6classes/Ventricles_{size}_{vent}'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

for img, filename in tqdm(zip(dataset, mask_paths), total=len(mask_paths)):
    uid = filename.split('/')[-1][:5]
    mask = img['label'].data[0].numpy()
    nrrd.write(f'{out_dir}/{uid}.nrrd', mask)

100%|██████████| 197/197 [01:40<00:00,  1.95it/s]


In [10]:
filenames = glob('../dataset/All/Ventricles_256_4/*.nrrd')
sizes = np.zeros((len(filenames), 3))
sizes.shape

(1090, 3)

In [11]:
for i, filename in enumerate(tqdm(filenames)):
    mask = nrrd.read(filename)[0]
    indices = np.argwhere(mask)
    center = ndimage.measurements.center_of_mass(mask)
    for j in range(3):
        sizes[i, j] = int(2 * max(indices[:, j].max() - center[j], center[j] - indices[:, j].min())) + 1 

100%|██████████| 1090/1090 [21:16<00:00,  1.17s/it]


In [12]:
for j in range(3):
    print(sizes[:, j].min(), sizes[:, j].max())

12.0 39.0
10.0 60.0
26.0 72.0


In [ ]:
# 1st vent
# 30.0 65.0
# 70.0 144.0
# 46.0 96.0

# 3rd vent
# 4.0 20.0
# 23.0 41.0
# 15.0 32.0

# 4th vent
# 12.0 39.0
# 10.0 60.0
# 26.0 72.0